
Authors: W.T. Chung, M. Saboo, C.-Z. Tan. For modifying the LVIS dataset to only consider food objects.

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment2/'
FOLDERNAME = 'cs231n/project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))


Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

# Small COCO Dataset for Milestone

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/coco_annotations/instances_val2017.json')
 
# returns JSON object as
# a dictionary
anno= json.load(f)

#Convert category info and image info into df
coco_cat = pd.DataFrame(anno['categories'])
coco_img = pd.DataFrame(anno['images'])
coco_anno = pd.DataFrame(anno['annotations'])

# img_id = ['000000512929', '000000056288', '000000322211', '000000488075', '000000034205', '000000213224', '000000356248', '000000480275']
# Restrict annotations to small set of images
img_id = [512929, 56288, 322211, 488075, 34205, 213224, 356248, 480275]
coco_anno = coco_anno[coco_anno['image_id'].isin(img_id)]
coco_img = coco_img[coco_img['id'].isin(img_id)]

#Register filenames
coco_anno_img_id = coco_anno['image_id']
coco_anno_file_name = ['/content/drive/MyDrive/cs231n/project/small_data/'+str(i).zfill(12)+".jpg" for i in coco_anno_img_id]
coco_anno.insert(0,'filename',coco_anno_file_name)

#Print example of annotation needed for detectron to read
print(coco_anno.head(3))

#Convert to json needed for detectron
with open('/content/drive/MyDrive/cs231n/project/small_data.json', 'w') as f:
    json.dump({'annotations':coco_anno.to_dict('records'),'categories':coco_cat.to_dict('records'),'images':coco_img.to_dict('records')}, f)
    

                                               filename  \
4304  /content/drive/MyDrive/cs231n/project/small_da...   
4313  /content/drive/MyDrive/cs231n/project/small_da...   
4315  /content/drive/MyDrive/cs231n/project/small_da...   

                                           segmentation       area  iscrowd  \
4304  [[626.74, 144.2, 629.44, 143.8, 631.19, 144.47...  348.60155        0   
4313  [[635.8, 156.17, 639.61, 156.38, 639.61, 186.0...  222.89330        0   
4315  [[411.0, 183.82, 411.08, 179.91, 414.4, 179.56...   40.52385        0   

      image_id                           bbox  category_id     id  
4304    488075  [620.09, 143.8, 12.99, 40.98]           44  86166  
4313    488075  [630.09, 156.17, 9.52, 29.84]           44  92505  
4315    488075    [411.0, 179.56, 5.11, 11.5]           44  93816  


In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/small_data.json')
 
# returns JSON object as
# a dictionary
print(json.load(f))

{'annotations': [{'filename': '/content/drive/MyDrive/cs231n/project/small_data/000000488075.jpg', 'segmentation': [[626.74, 144.2, 629.44, 143.8, 631.19, 144.47, 631.33, 148.65, 632.0, 150.54, 632.13, 155.8, 631.87, 158.09, 633.08, 160.65, 633.08, 164.7, 630.52, 166.99, 629.76, 184.49, 623.76, 184.78, 620.09, 184.05, 620.68, 172.91, 620.97, 166.61, 622.0, 160.17, 624.2, 158.11, 625.66, 154.3, 626.39, 144.19]], 'area': 348.60155000000043, 'iscrowd': 0, 'image_id': 488075, 'bbox': [620.09, 143.8, 12.99, 40.98], 'category_id': 44, 'id': 86166}, {'filename': '/content/drive/MyDrive/cs231n/project/small_data/000000488075.jpg', 'segmentation': [[635.8, 156.17, 639.61, 156.38, 639.61, 186.01, 630.09, 183.9, 630.3, 166.75, 635.17, 163.79]], 'area': 222.89330000000098, 'iscrowd': 0, 'image_id': 488075, 'bbox': [630.09, 156.17, 9.52, 29.84], 'category_id': 44, 'id': 92505}, {'filename': '/content/drive/MyDrive/cs231n/project/small_data/000000488075.jpg', 'segmentation': [[411.0, 183.82, 411.08,

# Small Dataset for LVIS

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/lvis_v1_val.json')
 
# returns JSON object as
# a dictionary
anno= json.load(f)

#Convert category info and image info into df
lvis_cat = pd.DataFrame(anno['categories'])
lvis_img = pd.DataFrame(anno['images'])
lvis_anno = pd.DataFrame(anno['annotations'])

# img_id = ['000000512929', '000000056288', '000000322211', '000000488075', '000000034205', '000000213224', '000000356248', '000000480275']
# Restrict annotations to small set of images
img_id = [512929, 56288, 322211, 488075, 34205, 213224, 356248, 480275]
lvis_anno = lvis_anno[lvis_anno['image_id'].isin(img_id)]
lvis_img = lvis_img[lvis_img['id'].isin(img_id)]

#Register filenames
lvis_anno_img_id = lvis_anno['image_id']
lvis_anno_file_name = [str(i).zfill(12)+".jpg" for i in lvis_anno_img_id]
lvis_anno.insert(0,'filename',lvis_anno_file_name)
lvis_anno.insert(0,'iscrowd',0)

lvis_img_img_id = lvis_img['id']
lvis_img_file_name = [str(i).zfill(12)+".jpg" for i in lvis_img_img_id]
lvis_img.insert(0,'file_name',lvis_img_file_name)
lvis_img.insert(0,'iscrowd',0)
#Print example of annotation needed for detectron to read
print(lvis_anno.head(3))

#load food cats, this csv was preprocessed beforehand
food_cats = pd.read_csv('/content/drive/MyDrive/cs231n/project/lvis_food_categories.csv')
#extract lvis ids for food objects
food_id = lvis_cat["id"][lvis_cat['name'].isin(food_cats['name'])]


#build dictionary to convert lvis category IDs to new ids
lvisIDtoNewID = pd.DataFrame(food_id.values.T,columns = ['lvis_id']).to_dict()['lvis_id']
lvisIDtoNewID = dict((v,k) for k,v in lvisIDtoNewID.items())
#Create new annotations for new classifier
lvis_anno = lvis_anno[lvis_anno['category_id'].isin(food_id)]
lvis_cat = lvis_cat[lvis_cat['id'].isin(food_id)]
for i in food_id:
  lvis_anno['category_id'] = lvis_anno['category_id'].replace(i,lvisIDtoNewID[i])
  lvis_cat['id'] = lvis_cat['id'].replace(i,lvisIDtoNewID[i])


#Convert to json needed for detectron
with open('/content/drive/MyDrive/cs231n/project/small_lvis_data.json', 'w') as f:
    json.dump({'annotations':lvis_anno.to_dict('records'),'categories':lvis_cat.to_dict('records'),'images':lvis_img.to_dict('records')}, f)
    

    iscrowd          filename    area  id  \
22        0  000000512929.jpg   49.87  23   
23        0  000000512929.jpg  295.39  24   
24        0  000000512929.jpg  280.17  25   

                                         segmentation  image_id  \
22  [[375.29, 185.23, 373.84, 181.32, 376.52, 178....    512929   
23  [[284.87, 566.74, 296.42, 577.82, 303.65, 588....    512929   
24  [[272.0, 500.49, 284.61, 498.48, 303.81, 497.3...    512929   

                              bbox  category_id  
22    [373.84, 178.43, 8.66, 8.04]          385  
23  [284.87, 558.56, 28.41, 30.33]           25  
24   [269.99, 491.72, 57.76, 8.77]           25  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# LVIS food set annotations

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/lvis_v1_val.json')
 
# returns JSON object as
# a dictionary
anno= json.load(f)


#Convert category info and image info into df
lvis_cat = pd.DataFrame(anno['categories'])
lvis_img = pd.DataFrame(anno['images'])
lvis_anno = pd.DataFrame(anno['annotations'])

#load food cats, this csv was preprocessed beforehand
food_cats = pd.read_csv('/content/drive/MyDrive/cs231n/project/lvis_food_categories.csv')
#extract lvis ids for food objects
food_id = lvis_cat["id"][lvis_cat['name'].isin(food_cats['name'])]

# Restrict annotations to images with food
lvis_anno = lvis_anno[lvis_anno['category_id'].isin(food_id)]
img_id = pd.unique(lvis_anno['image_id'])
lvis_img = lvis_img[lvis_img['id'].isin(img_id)]

#Register filenames
lvis_anno_img_id = lvis_anno['image_id']
lvis_anno_file_name = [str(i).zfill(12)+".jpg" for i in lvis_anno_img_id]
lvis_anno.insert(0,'filename',lvis_anno_file_name)
lvis_anno.insert(0,'iscrowd',0)

lvis_img_img_id = lvis_img['id']
lvis_img_file_name = [str(i).zfill(12)+".jpg" for i in lvis_img_img_id]
lvis_img.insert(0,'file_name',lvis_img_file_name)
lvis_img.insert(0,'iscrowd',0)
#Print example of annotation needed for detectron to read
print(lvis_anno.head(3))




#build dictionary to convert lvis category IDs to new ids
lvisIDtoNewID = pd.DataFrame(food_id.values.T,columns = ['lvis_id']).to_dict()['lvis_id']
lvisIDtoNewID = dict((v,k) for k,v in lvisIDtoNewID.items())
#Create new annotations for new classifier
lvis_cat = lvis_cat[lvis_cat['id'].isin(food_id)]
for i in food_id:
  lvis_anno['category_id'] = lvis_anno['category_id'].replace(i,lvisIDtoNewID[i])
  lvis_cat['id'] = lvis_cat['id'].replace(i,lvisIDtoNewID[i])

from sklearn.model_selection import train_test_split

val_anno , test_anno = train_test_split(lvis_anno, train_size=0.5,random_state=42)
val_img = lvis_img[lvis_img['id'].isin(val_anno['image_id'])]
test_img = lvis_img[lvis_img['id'].isin(test_anno['image_id'])]
val_out = {'annotations':val_anno.to_dict('records'),'categories':lvis_cat.to_dict('records'),'images':val_img.to_dict('records')}
test_out = {'annotations':test_anno.to_dict('records'),'categories':lvis_cat.to_dict('records'),'images':test_img.to_dict('records')}

#Convert to json needed for detectron
with open('/content/drive/MyDrive/cs231n/project/detectron_lvis_val_data.json', 'w') as f:
    json.dump(val_out, f)

with open('/content/drive/MyDrive/cs231n/project/detectron_lvis_test_data.json', 'w') as f:
    json.dump(test_out, f)
    

    iscrowd          filename    area  id  \
23        0  000000512929.jpg  295.39  24   
24        0  000000512929.jpg  280.17  25   
25        0  000000512929.jpg  384.74  26   

                                         segmentation  image_id  \
23  [[284.87, 566.74, 296.42, 577.82, 303.65, 588....    512929   
24  [[272.0, 500.49, 284.61, 498.48, 303.81, 497.3...    512929   
25  [[311.09, 547.25, 311.09, 555.23, 308.14, 557....    512929   

                              bbox  category_id  
23  [284.87, 558.56, 28.41, 30.33]           25  
24   [269.99, 491.72, 57.76, 8.77]           25  
25  [283.76, 515.29, 27.33, 42.05]           25  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# COCO Val Annotations for baseline

In [ ]:
f = open('/content/drive/MyDrive/cs231n/project/detectron_lvis_test_data.json')

In [ ]:
lvis_anno= json.load(f)

#Convert category info and image info into df
lvis_cat = pd.DataFrame(lvis_anno['categories'])
lvis_img = pd.DataFrame(lvis_anno['images'])
lvis_anno = pd.DataFrame(lvis_anno['annotations'])

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/coco_annotations/instances_val2017.json')
 
# returns JSON object as
# a dictionary
anno= json.load(f)

#Convert category info and image info into df
coco_cat = pd.DataFrame(anno['categories'])
coco_img = pd.DataFrame(anno['images'])
coco_anno = pd.DataFrame(anno['annotations'])

f2 = open('/content/drive/MyDrive/cs231n/project/coco_annotations/instances_train2017.json')
 
# returns JSON object as
# a dictionary
anno2= json.load(f2)

#Convert category info and image info into df
coco_img2 = pd.DataFrame(anno2['images'])
coco_anno2 = pd.DataFrame(anno2['annotations'])
coco_anno = pd.concat([coco_anno,coco_anno2],ignore_index=True)
coco_img = pd.concat([coco_img,coco_img2],ignore_index=True)


# img_id = ['000000512929', '000000056288', '000000322211', '000000488075', '000000034205', '000000213224', '000000356248', '000000480275']
# Restrict annotations to small set of images
img_id = lvis_img.id.unique()
coco_anno = coco_anno[coco_anno['image_id'].isin(img_id)]
coco_img = coco_img[coco_img['id'].isin(coco_anno.image_id.unique())]

#Register filenames
coco_anno_img_id = coco_anno['image_id']
coco_anno_file_name = [str(i).zfill(12)+".jpg" for i in coco_anno_img_id]
coco_anno.insert(0,'filename',coco_anno_file_name)

#Print example of annotation needed for detectron to read
print(coco_anno.head(3))

#Convert to json needed for detectron
with open('/content/drive/MyDrive/cs231n/project/coco_test_data.json', 'w') as f:
    json.dump({'annotations':coco_anno.to_dict('records'),'categories':coco_cat.to_dict('records'),'images':coco_img.to_dict('records')}, f)
    

            filename                                       segmentation  \
31  000000037777.jpg  [[110.39, 135.78, 110.39, 127.62, 110.01, 119....   
32  000000491216.jpg  [[282.07, 241.93, 276.44, 218.14, 285.2, 211.8...   
41  000000491216.jpg  [[215.47, 243.67, 211.57, 237.33, 208.65, 233....   

          area  iscrowd  image_id                            bbox  \
31    88.52115        0     37777    [102.49, 118.47, 7.9, 17.31]   
32  1164.92650        0    491216   [269.55, 180.58, 29.42, 62.6]   
41  1490.46210        0    491216  [207.18, 171.48, 38.54, 74.14]   

    category_id     id  
31           64  22328  
32           64  22674  
41           64  26288  


# LVIS Train Annotations

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/cs231n/project/lvis_v1_train.json')
 
# returns JSON object as
# a dictionary
anno= json.load(f)


#Convert category info and image info into df
lvis_cat = pd.DataFrame(anno['categories'])
lvis_img = pd.DataFrame(anno['images'])
lvis_anno = pd.DataFrame(anno['annotations'])

#load food cats, this csv was preprocessed beforehand
food_cats = pd.read_csv('/content/drive/MyDrive/cs231n/project/lvis_food_categories.csv')
#extract lvis ids for food objects
food_id = lvis_cat["id"][lvis_cat['name'].isin(food_cats['name'])]

# Restrict annotations to images with food
lvis_anno = lvis_anno[lvis_anno['category_id'].isin(food_id)]
img_id = pd.unique(lvis_anno['image_id'])
lvis_img = lvis_img[lvis_img['id'].isin(img_id)]

#Register filenames
lvis_anno_img_id = lvis_anno['image_id']
lvis_anno_file_name = [str(i).zfill(12)+".jpg" for i in lvis_anno_img_id]
lvis_anno.insert(0,'filename',lvis_anno_file_name)
lvis_anno.insert(0,'iscrowd',0)

lvis_img_img_id = lvis_img['id']
lvis_img_file_name = [str(i).zfill(12)+".jpg" for i in lvis_img_img_id]
lvis_img.insert(0,'file_name',lvis_img_file_name)
lvis_img.insert(0,'iscrowd',0)
#Print example of annotation needed for detectron to read
print(lvis_anno.head(3))




#build dictionary to convert lvis category IDs to new ids
lvisIDtoNewID = pd.DataFrame(food_id.values.T,columns = ['lvis_id']).to_dict()['lvis_id']
lvisIDtoNewID = dict((v,k) for k,v in lvisIDtoNewID.items())
#Create new annotations for new classifier
lvis_cat = lvis_cat[lvis_cat['id'].isin(food_id)]
for i in food_id:
  lvis_anno['category_id'] = lvis_anno['category_id'].replace(i,lvisIDtoNewID[i])
  lvis_cat['id'] = lvis_cat['id'].replace(i,lvisIDtoNewID[i])


train_out = {'annotations':lvis_anno.to_dict('records'),'categories':lvis_cat.to_dict('records'),'images':lvis_img.to_dict('records')}

#Convert to json needed for detectron
with open('/content/drive/MyDrive/cs231n/project/detectron_lvis_train_data.json', 'w') as f:
    json.dump(train_out, f)


    iscrowd          filename                            bbox  category_id  \
13        0  000000029358.jpg   [136.18, 296.18, 90.5, 86.12]          387   
80        0  000000484573.jpg   [229.65, 305.1, 44.73, 82.54]          154   
81        0  000000484573.jpg  [559.89, 208.88, 60.54, 62.71]          154   

    image_id  id                                       segmentation     area  
13     29358  14  [[197.24, 322.94, 185.56, 312.48, 177.78, 307....  3777.83  
80    484573  81  [[246.89, 377.38, 251.55, 379.71, 256.21, 383....  2674.89  
81    484573  82  [[619.66, 240.04, 620.43, 232.16, 615.82, 229....  2251.70  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
